## Harmony Py Library
### Job Pause/Resume Example

In [ ]:
import sys
import helper
helper.install_project_and_dependencies('..')

from harmony import BBox, Client, Collection, Request, Environment

In [ ]:
harmony_client = Client(env=Environment.UAT)  # assumes .netrc usage

collection = Collection(id='C1234724470-POCLOUD')
request = Request(
    collection=collection,
    spatial=BBox(0, -45, 180, 45),
    max_results=101
)

#### submit an async request for processing and return the job_id, big requests get automatically paused after generating a preview of the results

In [ ]:
job_id = harmony_client.submit(request)

#### checking the status of the job we see that it is 'previewing'

In [ ]:
harmony_client.status(job_id)

#### 'wait_for_processing()' will wait while the job is in the 'previewing' state then warns that the job is paused before exiting

In [ ]:
harmony_client.wait_for_processing(job_id, show_progress=True)

#### checking the status we see that the job is paused

In [ ]:
harmony_client.status(job_id)

#### 'result_json()' will not wait for paused jobs and just returns any available results.

In [ ]:
results = harmony_client.download_all(job_id, directory='/tmp', overwrite=True)
count = 0
for r in results:
    count += 1
print(f'Got {count} results')

/tmp/5405885_20020704124006-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0_subsetted.nc4


#### we can resume the job with 'resume()'

In [ ]:
harmony_client.resume(job_id)

#### checking the status we see that the job is running again

In [ ]:
harmony_client.status(job_id)

#### we can pause the job with 'pause()'.

In [ ]:
harmony_client.pause(job_id)

#### checking the status we see that the job is paused again

In [ ]:
harmony_client.status(job_id)

#### We can resume the job again

In [ ]:
harmony_client.resume(job_id)

#### 'wait_for_processing()' will show resumed progress

In [ ]:
harmony_client.wait_for_processing(job_id, show_progress=True)

#### 'download_all()' now has access to the full results

In [ ]:
results = harmony_client.download_all(job_id, directory='/tmp', overwrite=True)
count = 0
for r in results:
    count += 1
print(f'Got {count} results')

#### Attempting to pause a completed job will result in an error

In [ ]:
try: 
    harmony_client.pause(job_id)
except Exception as e:
    print(e)

#### Attempting to resume a completed job will also result in an error

In [ ]:
try: 
    harmony_client.resume(job_id)
except Exception as e:
    print(e)

#### we can use the 'skip_preview' parameter to tell Harmony to skip the auto-pause/preview and just start running

In [ ]:
harmony_client = Client(env=Environment.UAT)  # assumes .netrc usage

collection = Collection(id='C1234724470-POCLOUD')
request = Request(
    collection=collection,
    spatial=BBox(0, -45, 180, 45),
    max_results=101,
    skip_preview=True
)

#### submit an async request for processing and return the job_id, big requests get automatically paused after generating a preview of the results

In [ ]:
job_id = harmony_client.submit(request)

#### checking the status we see that the job is running

In [ ]:
harmony_client.status(job_id)

#### we can now use'wait_for_processing()' to wait until the job completes

In [ ]:
harmony_client.wait_for_processing(job_id, show_progress=True)

### Cancel Job Example

#### Submit a job to be canceled

In [ ]:
harmony_client = Client(env=Environment.UAT)  # assumes .netrc usage

collection = Collection(id='C1234724470-POCLOUD')
request = Request(
    collection=collection,
    spatial=BBox(0, -45, 180, 45),
    max_results=101
)
job_id = harmony_client.submit(request)
harmony_client.status(job_id)

#### Cancel the job and check for job status

In [ ]:
harmony_client.cancel(job_id)

In [ ]:
harmony_client.status(job_id)